## Dataset Preparation
1. Download and Explore the Dataset:  
- Download the dataset from the provided Codalab competition link.
- Explore the data structure, including image dimensions, file formats, and corresponding labels.

2. Data Augmentation:  
- Use augmentations like rotations, flips, and brightness adjustments to increase the dataset's variety. Leverage libraries like `torchvision.transforms` or `Albumentations`.

3. Dataset Class:  
- Create a PyTorch dataset class that loads images and masks, applies augmentations, and prepares data for model training:

In [ ]:
import os

from PIL import Image
import numpy as np

from torch.utils.data import Dataset
from torchvision.transforms import Compose, ToTensor, Normalize

class HaulRoadDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None):
        """
        Args:
            images_dir (str): Path to the directory containing images.
            masks_dir (str): Path to the directory containing masks.
            transform (callable, optional): Transformation to apply to both images and masks.
        """
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.images = sorted(os.listdir(images_dir))
        self.masks = sorted(os.listdir(masks_dir))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Load image and mask
        img_path = os.path.join(self.images_dir, self.images[idx])
        mask_path = os.path.join(self.masks_dir, self.masks[idx])
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # Grayscale for binary mask

        # Convert mask to binary (0 and 1)
        mask = np.array(mask) / 255.0  # Scale mask to range [0, 1]
        mask = Image.fromarray((mask > 0.5).astype(np.uint8))  # Threshold mask to binary

        # Apply transformations
        if self.transform:
            transformed = self.transform(image=np.array(image), mask=np.array(mask))
            image = transformed['image']
            mask = transformed['mask']

        # Convert to tensors
        image = ToTensor()(image)
        mask = ToTensor()(mask)

        return image, mask


## Model Selection
1. Backbone Model:  
Use a segmentation model like U-Net, DeepLabV3, or SegFormer. Libraries like segmentation_models_pytorch can help simplify this step.

2. Customize for Road Segmentation:  
Choose an appropriate backbone (e.g., ResNet, EfficientNet) and adjust output classes for binary segmentation (road vs. background).

3. PyTorch Lightning Module:  
Wrap your model into a Lightning module for better structure:

In [2]:
import pytorch_lightning as pl
import torch.nn as nn
import torch.optim as optim

class RoadSegmentationModel(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-3):
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.criterion = nn.BCEWithLogitsLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, masks = batch
        outputs = self(images)
        loss = self.criterion(outputs, masks)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.learning_rate)


In [4]:
from scipy.ndimage import label

def calculate_road_size(mask, pixel_to_meter_ratio):
    labeled_array, num_features = label(mask)
    road_sizes = []
    for i in range(1, num_features + 1):
        component = (labeled_array == i).astype(np.uint8)
        road_size = component.sum() * pixel_to_meter_ratio**2
        road_sizes.append(road_size)
    return road_sizes


## Training

1. Data Loaders:  
- Create PyTorch data loaders for train, validation, and test splits.

2. Training Loop:  
- Use PyTorch Lightning’s Trainer to handle the training loop and evaluation:

In [5]:
import segmentation_models_pytorch as smp

# Define the base segmentation model
base_model = smp.Unet(
    encoder_name="resnet34",  # Choose your backbone
    encoder_weights="imagenet",  # Pre-trained on ImageNet
    in_channels=3,  # Input channels (e.g., RGB images)
    classes=1,  # Output channels (binary segmentation)
)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /home/asrulsibaoel/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [03:20<00:00, 436kB/s]


In [ ]:
from torchvision.transforms import Normalize

# Define augmentations or transformations (if any)
transform = Compose([
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize images
])

# Create dataset instances
train_dataset = HaulRoadDataset(
    images_dir="data/train/images",
    masks_dir="data/train/masks",
    transform=transform
)
val_dataset = HaulRoadDataset(
    images_dir="data/val/images",
    masks_dir="data/val/masks",
    transform=transform
)


In [ ]:
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader

trainer = Trainer(max_epochs=50, gpus=1)
road_segmentation_model = RoadSegmentationModel(model=base_model)


# Define your data loaders (train and validation)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)

# Instantiate the PyTorch Lightning trainer
trainer = Trainer(max_epochs=50, gpus=1)  # Adjust gpus to 0 if no GPU is available

# Fit the model
trainer.fit(road_segmentation_model, train_dataloader, val_dataloader)

